In [1]:
import os
from tqdm.notebook import tqdm
import pandas as pd
import glob

In [2]:
from scripts import image_attributes

In [3]:
# Settings
image_attributes.save_color_images = False
image_attributes.save_lbp_images = False
image_attributes.save_gradient_images = False

Create a dataset

In [4]:
df_dataset = pd.DataFrame()
folders = [r"./input"]
for folder in tqdm(folders, desc="All folders progress"):
    for file_path in tqdm(glob.glob(os.path.join(folder, "*.jpg")), desc="Current folder progress"):
        feats, _, _, _ = image_attributes.get_features(file_path)
        df_dataset = pd.concat([df_dataset, feats], ignore_index=True)

All folders progress:   0%|          | 0/1 [00:00<?, ?it/s]

Current folder progress:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
df_dataset

,path,source_w,source_h,face_index,obj_score,class_score,x1,y1,x2,y2,...,SOBEL_BIN_bbox_16,SOBEL_BIN_bbox_17,SOBEL_BIN_bbox_18,SOBEL_BIN_bbox_19,SOBEL_BIN_bbox_20,SOBEL_BIN_bbox_21,SOBEL_BIN_bbox_22,SOBEL_BIN_bbox_23,SOBEL_BIN_bbox_24,SOBEL_BIN_bbox_25
0,./input\sample1.jpg,1024,1024,0,0.999962,0.999982,113,77,281,337,...,0.029623,0.028113,0.021226,0.024717,0.023774,0.020377,0.020000,0.019434,0.020094,0.018868
1,./input\sample2.jpg,457,450,0,0.999709,0.999919,164,90,291,213,...,0.034372,0.034693,0.040154,0.039190,0.032766,0.031481,0.037906,0.029553,0.026662,0.023129
2,./input\sample3.jpg,149,129,0,0.933927,0.997976,124,31,313,415,...,0.022685,0.024389,0.017571,0.036256,0.016653,0.018620,0.014555,0.013867,0.028946,0.009638


In [6]:
df_dataset.to_csv("min_epsilon_dataset.csv")

___